# 時系列画像分類器 - ハイパーパラメータ最適化 (Google Colab)

このノートブックは、Google Colab環境で時系列画像分類器のハイパーパラメータ最適化を実行するためのものです。

## 実行前の準備

1. Google Driveにプロジェクトフォルダをアップロードしておいてください
2. プロジェクトフォルダは `/content/drive/MyDrive/Time_Series_Classifier` に配置されている想定です
3. データファイルは `data` ディレクトリに配置されている必要があります

## 1. 環境設定とマウント

Google Driveをマウントし、プロジェクトディレクトリに移動します。

In [ ]:
# Google Driveをマウント
from google.colab import drive
drive.mount('/content/drive')

# プロジェクトディレクトリに移動
import os
project_root = '/content/drive/MyDrive/Time_Series_Classifier'
os.chdir(project_root)
print(f"カレントディレクトリ: {os.getcwd()}")

Mounted at /content/drive
/content/drive/MyDrive/NFNet_Classifier_pretrained


## 2. 必要なライブラリのインストール

時系列画像分類器の学習に必要なライブラリをインストールします。

In [ ]:
# # 既存のパッケージをアンインストール
# ! pip uninstall -y lightning pytorch-lightning optuna optuna-integration

# # ランタイムキャッシュを消去
# import sys
# for mod in list(sys.modules.keys()):
#     if any(x in mod for x in ['lightning', 'pytorch_lightning', 'optuna']):
#         sys.modules.pop(mod, None)
#         print(f"Removed from sys.modules: {mod}")

## 3. ハイパーパラメータ最適化の実行

Optunaを使用して、時系列画像分類モデルのハイパーパラメータを最適化します。

In [ ]:
# 環境変数を設定（最適化スクリプトが参照する設定ファイルのパス）
import os

# Google Colab用の設定ファイルパスを指定
base_config_path = '/content/drive/MyDrive/Time_Series_Classifier/configs/config_for_google_colab.yaml'
tuning_config_path = '/content/drive/MyDrive/Time_Series_Classifier/tuning/config_for_google_colab.yaml'

# 環境変数として設定
os.environ['BASE_CONFIG_PATH'] = base_config_path
os.environ['TUNING_CONFIG_PATH'] = tuning_config_path

print(f"BASE_CONFIG_PATH: {os.environ['BASE_CONFIG_PATH']}")
print(f"TUNING_CONFIG_PATH: {os.environ['TUNING_CONFIG_PATH']}")

# 設定ファイルの存在確認
for path, name in [(base_config_path, "ベース設定"), (tuning_config_path, "チューニング設定")]:
    if os.path.exists(path):
        print(f"✅ {name}ファイルが見つかりました: {path}")
    else:
        print(f"❌ {name}ファイルが見つかりません: {path}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.0/819.0 kB 50.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 961.5/961.5 kB 60.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.6/386.6 kB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 129.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 101.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 45.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## 4. 最適化の実行

設定されたパラメータ範囲でハイパーパラメータの最適化を開始します。

In [ ]:
# 設定ファイルの読み込みと確認
import yaml

config_path = os.environ['TUNING_CONFIG_PATH']

try:
    with open(config_path, 'r', encoding='utf-8') as file:
        config = yaml.safe_load(file)
    
    print("=== 分類器設定ファイル内容 ===")
    print(f"データセット設定:")
    print(f"  - データセットA: {config.get('dataset_a_dir', 'N/A')}")
    print(f"  - データセットB: {config.get('dataset_b_dir', 'N/A')}")
    print(f"  - バッチサイズ: {config.get('batch_size', 'N/A')}")
    
    print(f"\nモデル設定:")
    print(f"  - アーキテクチャ: {config.get('model_name', 'N/A')}")
    print(f"  - クラス数: {config.get('num_classes', 'N/A')}")
    print(f"  - クラス名: {config.get('class_names', 'N/A')}")
    
    print(f"\n学習設定:")
    print(f"  - 最大エポック数: {config.get('max_epochs', 'N/A')}")
    print(f"  - 学習率: {config.get('learning_rate', 'N/A')}")
    
    print("\n設定ファイル読み込み成功✅")
    
except FileNotFoundError:
    print(f"❌ エラー: 設定ファイルが見つかりません: {config_path}")
except yaml.YAMLError as e:
    print(f"❌ YAML解析エラー: {e}")
except Exception as e:
    print(f"❌ 予期しないエラー: {e}")

Sat May 17 06:32:44 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   30C    P0             47W /  400W |       5MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

## 5. 最適化結果の確認

最適化完了後、ベストトライアルの情報を確認します。

In [ ]:
# データセットディレクトリの確認
import os

def check_dataset_structure(base_path, dataset_name):
    """データセットディレクトリの構造を確認"""
    print(f"\n=== {dataset_name} データセット確認 ===")
    
    if not os.path.exists(base_path):
        print(f"❌ データセットディレクトリが見つかりません: {base_path}")
        return False
    
    print(f"✅ データセットパス: {base_path}")
    
    # クラスディレクトリの確認
    try:
        class_dirs = [d for d in os.listdir(base_path) 
                     if os.path.isdir(os.path.join(base_path, d))]
        print(f"クラス数: {len(class_dirs)}")
        
        for class_dir in sorted(class_dirs):
            class_path = os.path.join(base_path, class_dir)
            image_count = len([f for f in os.listdir(class_path) 
                             if f.lower().endswith(('.png', '.jpg', '.jpeg'))])
            print(f"  - {class_dir}: {image_count} 画像")
            
    except Exception as e:
        print(f"❌ ディレクトリ読み込みエラー: {e}")
        return False
    
    return True

# 設定からデータセットパスを取得
dataset_a_path = config.get('dataset_a_dir')
dataset_b_path = config.get('dataset_b_dir')

# データセット確認
if dataset_a_path:
    check_dataset_structure(dataset_a_path, "データセットA")
    
if dataset_b_path:
    check_dataset_structure(dataset_b_path, "データセットB")

## 6. 最適化履歴の可視化

Optunaの最適化過程を可視化して、学習の進捗を確認します。

In [ ]:
# メイン実行: 時系列画像分類器の最適化と訓練
import sys
sys.path.append(PROJECT_ROOT)

from tuning.optimize import main

# 分類モデルの最適化実行
print("🚀 時系列画像分類モデルの最適化を開始します...")
print("📊 Optunaを使用してハイパーパラメータを最適化し、最良のモデルを見つけます")
print("⏰ 処理には時間がかかる場合があります...")

try:
    # 最適化実行
    main()
    print("✅ 分類モデルの最適化が完了しました!")
    
except Exception as e:
    print(f"❌ 最適化中にエラーが発生しました: {e}")
    import traceback
    traceback.print_exc()

環境変数 TUNING_CONFIG_PATH を設定しました: /content/drive/MyDrive/NFNet_Classifier_pretrained/tuning/config_for_google_colab.yaml
環境変数 BASE_CONFIG_PATH を設定しました: /content/drive/MyDrive/NFNet_Classifier_pretrained/configs/config_for_google_colab.yaml


In [ ]:
# 最適化結果の確認
import os
import json
from datetime import datetime

# 結果ディレクトリの確認
results_dir = os.path.join(PROJECT_ROOT, 'tuning')
checkpoints_dir = os.path.join(PROJECT_ROOT, 'checkpoints')

print("=== 最適化結果の確認 ===")

# 最新のベストパラメータファイルを探す
param_files = [f for f in os.listdir(results_dir) if f.startswith('best_params_') and f.endswith('.json')]
if param_files:
    latest_param_file = sorted(param_files)[-1]
    param_path = os.path.join(results_dir, latest_param_file)
    
    print(f"✅ 最新のベストパラメータファイル: {latest_param_file}")
    
    try:
        with open(param_path, 'r') as f:
            best_params = json.load(f)
        
        print("\n📊 最適化されたハイパーパラメータ:")
        for key, value in best_params.items():
            if key != 'trial_number':
                print(f"  - {key}: {value}")
        
        if 'trial_number' in best_params:
            print(f"\n🏆 ベスト試行番号: {best_params['trial_number']}")
            
    except Exception as e:
        print(f"❌ パラメータファイル読み込みエラー: {e}")
else:
    print("❌ ベストパラメータファイルが見つかりません")

# チェックポイントファイルの確認
if os.path.exists(checkpoints_dir):
    checkpoint_files = [f for f in os.listdir(checkpoints_dir) if f.endswith('.ckpt')]
    print(f"\n💾 保存されたチェックポイント数: {len(checkpoint_files)}")
    
    if checkpoint_files:
        print("最新のチェックポイント:")
        for ckpt in sorted(checkpoint_files)[-3:]:  # 最新3つを表示
            print(f"  - {ckpt}")
else:
    print("❌ チェックポイントディレクトリが見つかりません")

使用するチューニング設定ファイル: /content/drive/MyDrive/NFNet_Classifier_pretrained/tuning/config_for_google_colab.yaml
# Optunaによるハイパーパラメータ最適化の設定

# =====================================================================
# 事前最適化の設定 (参照用にコメントアウト)
# =====================================================================
# study:
#   name: "efficientnet_b4_optimization"  # モデル名を反映
#   direction: "maximize"  # val_f1を最大化
#   metric: "val_f1"      # 最適化する評価指標
#   n_trials: 40         # 試行回数
#   timeout: 172800       # タイムアウト (秒) - 48時間
#   tuning_max_epochs: 30 # チューニング用エポック数
#   is_post_training: false  # 事前最適化フラグ
#   # checkpoint_pathは事前最適化では使用しない


# =====================================================================
# 事後最適化の設定 (現在有効)
# =====================================================================
# 最適化の基本設定
study:
  name: "efficientnet_b4_post_training"  # 事後最適化用に名前を変更
  direction: maximize
  metric: val_f1
  n_trials: 40         # 試行回数
  timeout: 172800       # タイムアウト (秒) - 48時間
  tuning_max_e

## モデル評価とテスト

訓練されたモデルをテストデータセットで評価し、分類精度を測定します。

In [ ]:
# 訓練済みモデルの評価
from src.evaluate import main as evaluate_main

print("🔍 訓練済みモデルの評価を開始します...")
print("📈 テストデータセットで分類精度を測定します")

try:
    # モデル評価実行
    evaluate_main()
    print("✅ モデル評価が完了しました!")
    
    # 評価結果ファイルの確認
    logs_dir = os.path.join(PROJECT_ROOT, 'lightning_logs')
    if os.path.exists(logs_dir):
        version_dirs = [d for d in os.listdir(logs_dir) if d.startswith('version_')]
        if version_dirs:
            latest_version = sorted(version_dirs)[-1]
            print(f"📊 評価結果は {latest_version} ディレクトリに保存されました")
        
except Exception as e:
    print(f"❌ モデル評価中にエラーが発生しました: {e}")
    import traceback
    traceback.print_exc()

モデルモード: single
モデルアーキテクチャ名: efficientnet_b4
チェックポイントディレクトリ構造: /content/drive/MyDrive/NFNet_Classifier_pretrained/tuning/checkpoints/single/efficientnet_b4
段階的凍結解除: 無効
  差分学習率を使用します


## 結果の可視化

分類結果と学習履歴を可視化します。

In [ ]:
# 結果の可視化
from src.visualize import main as visualize_main

print("📈 分類結果と学習履歴の可視化を開始します...")

try:
    # 可視化実行
    visualize_main()
    print("✅ 可視化が完了しました!")
    
    # 生成された図表ファイルの確認
    figures_dir = os.path.join(PROJECT_ROOT, 'figures')
    if os.path.exists(figures_dir):
        figure_files = [f for f in os.listdir(figures_dir) 
                       if f.lower().endswith(('.png', '.jpg', '.jpeg', '.svg'))]
        print(f"📊 生成された図表: {len(figure_files)} 個")
        
        # 最新の図表を表示
        if figure_files:
            print("最新の図表:")
            for fig in sorted(figure_files)[-3:]:  # 最新3つを表示
                print(f"  - {fig}")
    
    print("\n🎯 分類精度や混同行列などの詳細な結果は、生成された図表をご確認ください")
    
except Exception as e:
    print(f"❌ 可視化中にエラーが発生しました: {e}")
    import traceback
    traceback.print_exc()

---現在ロードされているパッケージ---

Lightningバージョン: 2.5.1.post0
シングルモーダルモデル (SingleModalClassifier) を使用します

モデルクラスの継承クラス: (<class 'src.models.single_modal.SingleModalClassifier'>, <class 'lightning.pytorch.core.module.LightningModule'>, <class 'lightning.fabric.utilities.device_dtype_mixin._DeviceDtypeModuleMixin'>, <class 'lightning.pytorch.core.mixins.hparams_mixin.HyperparametersMixin'>, <class 'lightning.pytorch.core.hooks.ModelHooks'>, <class 'lightning.pytorch.core.hooks.DataHooks'>, <class 'lightning.pytorch.core.hooks.CheckpointHooks'>, <class 'torch.nn.modules.module.Module'>, <class 'object'>)
LightningModuleの型: <class 'lightning.pytorch.core.module.LightningModule'>
モデルがLightningModuleのサブクラスか: True

Optunaバージョン: 4.3.0
timmライブラリからモデル 'efficientnet_b4' を読み込みます...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/77.9M [00:00<?, ?B/s]

モデル 'efficientnet_b4' の読み込みに成功しました。
モデル 'efficientnet_b4' の特徴次元数: 1792
特徴抽出器の全パラメータを初期状態で学習可能に設定しました (差分学習率モード)。
分類ヘッドを定義しました (Dropout1: 0.3, Dropout2: 0.2)
モデルタイプ 'efficientnet_b4' のステージ構造を設定します...
  EfficientNetの 'blocks' モジュールから 7 個のステージ (ブロックグループ) を検出しました。
最終的なステージ数: 7 (入力に近い層（浅い）から出力に近い層（深い）の順に格納)
ステージごとのパラメータ数:
  Stage 0 (入力から第1層): 4,146 / 4,146 パラメータ (学習可能)
  Stage 1 (入力から第2層): 66,238 / 66,238 パラメータ (学習可能)
  Stage 2 (入力から第3層): 197,586 / 197,586 パラメータ (学習可能)
  Stage 3 (入力から第4層): 1,059,898 / 1,059,898 パラメータ (学習可能)
  Stage 4 (入力から第5層): 2,306,724 / 2,306,724 パラメータ (学習可能)
  Stage 5 (入力から第6層): 8,636,228 / 8,636,228 パラメータ (学習可能)
  Stage 6 (入力から第7層): 4,470,004 / 4,470,004 パラメータ (学習可能)

モデルインスタンス化成功: <class 'src.models.single_modal.SingleModalClassifier'>
モデルがLightningModuleのインスタンスか: True


## 🎉 実行完了

時系列画像分類器の訓練と評価が完了しました！

### 生成されたファイル
- **モデルファイル**: `checkpoints/` ディレクトリに保存
- **最適化結果**: `tuning/best_params_*.json` に保存
- **評価結果**: `lightning_logs/` ディレクトリに保存
- **可視化図表**: `figures/` ディレクトリに保存

### 次のステップ
1. 生成された図表で分類精度を確認
2. 最適化されたハイパーパラメータを確認
3. 必要に応じてさらなるファインチューニング

---

## 📋 追加機能・デバッグ

以下のセクションでは、追加的な分析やデバッグ用の機能を提供します。

### 学習履歴の詳細確認

TensorBoardログから学習の詳細な履歴を確認できます。

In [ ]:
# 学習履歴の詳細確認
import os
import pandas as pd
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator

def extract_tensorboard_metrics(log_dir):
    """TensorBoardログからメトリクスを抽出"""
    try:
        ea = EventAccumulator(log_dir)
        ea.Reload()
        
        print(f"📊 ログディレクトリ: {log_dir}")
        print(f"📈 利用可能なスカラーメトリクス: {ea.Tags()['scalars']}")
        
        # 主要メトリクスの抽出
        metrics = {}
        for tag in ea.Tags()['scalars']:
            scalar_events = ea.Scalars(tag)
            metrics[tag] = [(s.step, s.value) for s in scalar_events]
        
        return metrics
    
    except Exception as e:
        print(f"❌ TensorBoardログ読み込みエラー: {e}")
        return {}

# Lightning logsの確認
logs_dir = os.path.join(PROJECT_ROOT, 'lightning_logs')
if os.path.exists(logs_dir):
    version_dirs = [d for d in os.listdir(logs_dir) if d.startswith('version_')]
    
    if version_dirs:
        latest_version = sorted(version_dirs)[-1]
        version_path = os.path.join(logs_dir, latest_version)
        
        print(f"🔍 最新バージョン: {latest_version}")
        
        # メトリクス抽出
        metrics = extract_tensorboard_metrics(version_path)
        
        if metrics:
            print("\n📈 学習履歴サマリー:")
            for metric_name, values in metrics.items():
                if values:
                    final_value = values[-1][1]
                    print(f"  - {metric_name}: {final_value:.4f} (最終値)")
    else:
        print("❌ 学習履歴が見つかりません")
else:
    print("❌ Lightning logsディレクトリが見つかりません")

### メモリ断片化対策とGPUメモリ確保

- 大きなモデルを実行する前にキャッシュをクリアし、GPUメモリを確保します
- メモリ断片化を防ぐために `expandable_segments:True` を設定します

In [ ]:
# GPUキャッシュをクリアしてメモリを確保
import torch
import gc

# メモリ解放
gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    print(f"GPUメモリを解放しました。使用可能: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")

# メモリ断片化対策
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'
print(f"PYTORCH_CUDA_ALLOC_CONF set to: {os.environ['PYTORCH_CUDA_ALLOC_CONF']}")

# Google Colab用の最適化スクリプトを実行 (環境変数を読み込む)
! python /content/drive/MyDrive/NFNet_Classifier_pretrained/tuning/optimize_for_google_colab.py

Streaming output truncated to the last 5000 lines.
Validation DataLoader 0:  49% 1040/2110 [01:35<01:38, 10.83it/s]
Validation DataLoader 0:  50% 1060/2110 [01:37<01:37, 10.82it/s]
Epoch 30:  79% 3320/4220 [31:32<08:33,  1.75it/s, train_loss_step=0.534]
Validation DataLoader 0:  23% 120/528 [00:30<01:45,  3.88it/s]
Validation DataLoader 0:  52% 1100/2110 [01:41<01:33, 10.79it/s]
Validation DataLoader 0:  53% 1120/2110 [01:44<01:32, 10.76it/s]
Validation DataLoader 0:  27% 140/528 [00:36<01:40,  3.86it/s]
Validation DataLoader 0:  54% 1140/2110 [01:46<01:30, 10.69it/s]
Validation DataLoader 0:  55% 1160/2110 [01:48<01:29, 10.67it/s]
Validation DataLoader 0:  56% 1180/2110 [01:50<01:27, 10.65it/s]
Validation DataLoader 0:  30% 160/528 [00:41<01:34,  3.88it/s]
Epoch 30:  79% 3340/4220 [31:45<08:22,  1.75it/s, train_loss_step=0.346]
Validation DataLoader 0:  58% 1220/2110 [01:54<01:23, 10.64it/s]
Validation DataLoader 0:  34% 180/528 [00:46<01:30,  3.87it/s]
Validation DataLoader 0:  59% 1

## TensorBoardによる最適化過程の可視化

In [ ]:
# Colab用のTensorBoard拡張を読み込む
%load_ext tensorboard
# Optunaのログディレクトリを指定 (tuning/config_for_google_colab.yamlのoutput.log_dir)
%tensorboard --logdir=/content/drive/MyDrive/NFNet_Classifier_pretrained/tuning/logs

## Optunaスタディ結果の確認

In [ ]:
import optuna
import yaml
import os

# 環境変数から設定ファイルのパスを取得
tuning_config_path = os.environ.get('TUNING_CONFIG_PATH')
if not tuning_config_path or not os.path.exists(tuning_config_path):
    print(f"エラー: チューニング設定ファイルが見つかりません: {tuning_config_path}")
else:
    # Optuna設定ファイルを読み込む
    with open(tuning_config_path, 'r') as f:
        tuning_config = yaml.safe_load(f)

    # ストレージパスとスタディ名を取得 (設定ファイルから)
    storage_path = tuning_config['storage']['path'] # 設定ファイルに絶対パスが書かれている前提
    storage_url = f"sqlite:///{storage_path}"
    study_name = tuning_config['study']['name']

    # Optunaデータベース情報を表示（エラー発生時の対応のため）
    print(f"\n--- Optunaデータベース情報 ---")
    print(f"データベースファイル: {storage_path}")
    print(f"スタディ名: {study_name}")
    print(f"エラー発生時の削除コマンド: rm {storage_path}")

    # スタディ情報を表示
    try:
        study = optuna.load_study(study_name=study_name, storage=storage_url)
        print(f"スタディ '{study_name}' をロードしました。")
        print(f"完了したトライアル数: {len(study.trials)}")

        # 最良のトライアル情報を表示
        best_trial = study.best_trial
        print("\n--- 最良のトライアル --- ")
        print(f"  トライアル番号: {best_trial.number}")
        print(f"  評価値 ({tuning_config['study']['metric']}): {best_trial.value:.6f}")
        print("  パラメータ:")
        for key, value in best_trial.params.items():
            print(f"    {key}: {value}")

    except ValueError:
        print(f"スタディ '{study_name}' に完了したトライアルがまだありません。")
    except Exception as e:
        print(f"スタディのロードまたは結果の表示中にエラーが発生しました: {e}")
        print(f"エラー解決方法: データベースファイル '{storage_path}' を手動で削除してから再実行")

## Optuna結果の可視化

In [ ]:
import optuna
from optuna.visualization import (plot_optimization_history, plot_param_importances,
                                 plot_contour, plot_slice, plot_parallel_coordinate)
import yaml
import os
import plotly.io as pio

# Kaleidoを有効化 (静的画像エクスポート用)
pio.kaleido.scope.mathjax = None

# 環境変数から設定ファイルのパスを取得
tuning_config_path = os.environ.get('TUNING_CONFIG_PATH')
if not tuning_config_path or not os.path.exists(tuning_config_path):
    print(f"エラー: チューニング設定ファイルが見つかりません: {tuning_config_path}")
else:
    # Optuna設定ファイルを読み込む
    with open(tuning_config_path, 'r') as f:
        tuning_config = yaml.safe_load(f)

    # ストレージパスとスタディ名を取得 (設定ファイルから)
    storage_path = tuning_config['storage']['path'] # 設定ファイルに絶対パスが書かれている前提
    storage_url = f"sqlite:///{storage_path}"
    study_name = tuning_config['study']['name']

    try:
        study = optuna.load_study(study_name=study_name, storage=storage_url)
        print(f"スタディ '{study_name}' をロードしました。可視化を生成します...")

        # 完了したトライアルがあるか確認
        completed_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.COMPLETE]
        if not completed_trials:
            print("完了したトライアルがないため、可視化をスキップします。")
        else:
            # 可視化結果の保存先ディレクトリ (設定ファイルから取得)
            vis_save_dir = tuning_config['visualization']['save_dir']
            os.makedirs(vis_save_dir, exist_ok=True)
            print(f"可視化結果の保存先: {vis_save_dir}")

            # 最適化履歴
            try:
                fig_history = plot_optimization_history(study)
                fig_history.show()
                # fig_history.write_image(os.path.join(vis_save_dir, f"{study_name}_optimization_history.png"))
            except Exception as e:
                print(f"最適化履歴プロットの生成に失敗: {e}")

            # パラメータ重要度
            try:
                fig_importance = plot_param_importances(study)
                fig_importance.show()
                # fig_importance.write_image(os.path.join(vis_save_dir, f"{study_name}_param_importances.png"))
            except Exception as e:
                print(f"パラメータ重要度プロットの生成に失敗: {e}")

            # パラレル座標プロット
            try:
                fig_parallel = plot_parallel_coordinate(study)
                fig_parallel.show()
                # fig_parallel.write_image(os.path.join(vis_save_dir, f"{study_name}_parallel_coordinate.png"))
            except Exception as e:
                print(f"パラレル座標プロットの生成に失敗: {e}")

            # スライスプロット
            try:
                fig_slice = plot_slice(study)
                fig_slice.show()
                # fig_slice.write_image(os.path.join(vis_save_dir, f"{study_name}_slice.png"))
            except Exception as e:
                print(f"スライスプロットの生成に失敗: {e}")

            # コンタープロット (重要度上位2パラメータ)
            try:
                importances = optuna.importance.get_param_importances(study)
                top_params = list(importances.keys())[:2]
                if len(top_params) >= 2:
                    fig_contour = plot_contour(study, params=top_params)
                    fig_contour.show()
                    # fig_contour.write_image(os.path.join(vis_save_dir, f"{study_name}_contour.png"))
                elif len(completed_trials) > 0:
                     print("コンタープロットを生成するには、少なくとも2つの数値パラメータが必要です。")
            except ValueError as ve:
                print(f"コンタープロットの生成に失敗しました（{ve}）。試行回数やパラメータの種類を確認してください。")
            except Exception as e:
                print(f"コンタープロットの生成中に予期せぬエラーが発生しました: {e}")

    except ValueError:
        print(f"スタディ '{study_name}' に完了したトライアルがまだありません。")
    except Exception as e:
        print(f"Optuna結果の可視化中にエラーが発生しました: {e}")

## SQLiteデータベースのロック問題解決

Optunaは複数のトライアルを並列実行するとき、SQLiteデータベースのロックエラーが発生することがあります。以下の対策を実施します：

1. SQLiteデータベースのタイムアウト設定を追加
2. トライアル実行状態の確認
3. 必要に応じてデータベースのリセット

In [ ]:
# import os
# import yaml
# import sqlite3
# import optuna
# from datetime import datetime
# import shutil

# # 設定ファイルのパスを確認
# tuning_config_path = os.environ.get('TUNING_CONFIG_PATH')
# print(f"チューニング設定ファイル: {tuning_config_path}")

# # 設定ファイルを読み込む
# with open(tuning_config_path, 'r') as f:
#     tuning_config = yaml.safe_load(f)

# # SQLiteデータベースファイルのパスを取得
# storage_path = tuning_config['storage']['path']
# print(f"SQLiteデータベース: {storage_path}")

# # データベースのバックアップを作成（念のため）
# if os.path.exists(storage_path):
#     backup_timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
#     backup_path = f"{storage_path}.{backup_timestamp}.bak"
#     shutil.copy2(storage_path, backup_path)
#     print(f"データベースのバックアップを作成: {backup_path}")

# # SQLiteデータベースに直接接続してタイムアウト設定を確認
# try:
#     # タイムアウト設定を30秒に
#     conn = sqlite3.connect(storage_path, timeout=30000)
#     cursor = conn.cursor()

#     # DBのバージョン情報を表示
#     cursor.execute("SELECT sqlite_version();")
#     version = cursor.fetchone()
#     print(f"SQLite バージョン: {version[0]}")

#     # トライアルテーブルの情報を取得
#     cursor.execute("SELECT COUNT(*) FROM trials;")
#     total_trials = cursor.fetchone()[0]

#     cursor.execute("SELECT COUNT(*) FROM trials WHERE state = 1;")  # 1=RUNNING
#     running_trials = cursor.fetchone()[0]

#     cursor.execute("SELECT COUNT(*) FROM trials WHERE state = 2;")  # 2=COMPLETE
#     complete_trials = cursor.fetchone()[0]

#     cursor.execute("SELECT COUNT(*) FROM trials WHERE state = 3;")  # 3=FAIL
#     failed_trials = cursor.fetchone()[0]

#     conn.close()

#     print(f"トライアル総数: {total_trials}")
#     print(f"実行中のトライアル数: {running_trials}")
#     print(f"完了したトライアル数: {complete_trials}")
#     print(f"失敗したトライアル数: {failed_trials}")

# except sqlite3.Error as e:
#     print(f"SQLiteデータベースアクセスエラー: {e}")

### ロック問題が発生した際のトライアルリセット (必要時のみ実行)

複数のトライアルが並行して実行されるとSQLiteのロック競合が発生する可能性があります。以下のセルを使用して、状態が「実行中」のままになっているトライアルをリセットできます。

In [ ]:
# # このセルは必要な場合のみ実行してください（実行中状態のトライアルをリセット）
# reset_running_trials = False  # Trueに変更すると実行中のトライアルをリセットします

# if reset_running_trials:
#     try:
#         # SQLiteデータベースに直接接続してトライアルの状態をリセット
#         conn = sqlite3.connect(storage_path, timeout=30000)
#         cursor = conn.cursor()

#         # 実行中のトライアルの状態を「FAIL」に更新
#         cursor.execute("UPDATE trials SET state = 3 WHERE state = 1")  # 1=RUNNING, 3=FAIL
#         affected_rows = cursor.rowcount
#         conn.commit()
#         conn.close()

#         print(f"{affected_rows}件のトライアルを実行中から失敗状態にリセットしました")
#     except sqlite3.Error as e:
#         print(f"トライアルリセット中にエラーが発生しました: {e}")
# else:
#     print("トライアルのリセットはスキップされました。リセットする場合は 'reset_running_trials = True' に設定してください。")

### SQLiteタイムアウト設定の修正

SQLiteデータベースへのタイムアウト設定を自動的に行うため、optimize_for_google_colab.pyファイルを更新します。これによりロックエラーが減少します。

In [ ]:
# import re

# # optimize_for_google_colab.pyファイルのパス
# optimize_file_path = "/content/drive/MyDrive/NFNet_Classifier_pretrained/tuning/optimize_for_google_colab.py"

# # ファイル内容を読み込む
# with open(optimize_file_path, "r", encoding="utf-8") as file:
#     content = file.read()

# # タイムアウト設定を追加する正規表現パターン
# pattern = r"storage_url = f\"sqlite:///{tuning_config\['storage'\]\['path'\]}\""
# replacement = r"storage_url = f\"sqlite:///{tuning_config['storage']['path']}?timeout=30000\""

# # 置換を行う
# if re.search(pattern, content):
#     new_content = re.sub(pattern, replacement, content)

#     # 変更された内容を書き戻す
#     with open(optimize_file_path, "w", encoding="utf-8") as file:
#         file.write(new_content)
#     print("SQLiteタイムアウト設定が追加されました (timeout=30000ms)")
# else:
#     print("タイムアウト設定はすでに適用されているか、パターンが一致しませんでした")

### 並列実行設定の確認と更新

トライアルの並列実行数を確認・更新します。SQLiteのロック問題を完全に避けるには並列数を1にすることが推奨されますが、リスクを理解した上で高速化のために並列数を増やすこともできます。

In [ ]:
# # 現在の並列実行設定を確認
# current_n_jobs = tuning_config['parallel'].get('n_jobs', 1)
# print(f"現在の並列実行数: {current_n_jobs}")

# # 並列実行数を変更するかどうか
# change_n_jobs = False  # 変更する場合はTrueに設定
# new_n_jobs = 3  # 設定したい並列数

# if change_n_jobs:
#     # 設定ファイルを読み込み
#     with open(tuning_config_path, 'r', encoding='utf-8') as f:
#         tuning_config_content = f.read()

#     # n_jobs設定を変更
#     pattern = r"(parallel:\s*\n\s*n_jobs:\s*)\d+"
#     new_content = re.sub(pattern, f"\\1{new_n_jobs}", tuning_config_content)

#     # 変更を書き戻す
#     with open(tuning_config_path, 'w', encoding='utf-8') as f:
#         f.write(new_content)

#     print(f"並列実行数を {new_n_jobs} に変更しました")

#     # 設定ファイルを再読み込み
#     with open(tuning_config_path, 'r') as f:
#         tuning_config = yaml.safe_load(f)
#         print(f"新しい並列実行数: {tuning_config['parallel'].get('n_jobs', 1)}")
# else:
#     print("並列実行数の変更はスキップされました。変更する場合は 'change_n_jobs = True' に設定してください。")

## メモリ解放と最適化の再開

Optunaの最適化プロセスを再開します。SQLiteタイムアウト設定を適用した状態で実行することで、データベースのロックエラーを減少させることができます。

In [ ]:
# # GPUキャッシュをクリアしてメモリを確保
# import torch
# import gc
# import os

# # メモリ解放
# gc.collect()
# if torch.cuda.is_available():
#     torch.cuda.empty_cache()
#     print(f"GPUメモリを解放しました。使用可能: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")

# # メモリ断片化対策
# os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'
# print(f"PYTORCH_CUDA_ALLOC_CONF set to: {os.environ['PYTORCH_CUDA_ALLOC_CONF']}")

# # Google Colab用の最適化スクリプトを実行
# ! python /content/drive/MyDrive/NFNet_Classifier_pretrained/tuning/optimize_for_google_colab.py